In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import pandas as pd

# UNNECESSARY IMPORTS AS LIBRARIES ARE NEVER UTILIZED
#import numpy as np
#import matplotlib.pyplot as plt


# Import CRUD module
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Hardcode login credentials for the database
username = "aacuser"
password = "helloWorld"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    # Unique identifier as an H2 HTML element and client's logo
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.H2("JCROUCH332444937"),
    
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id="filter-type",
            options={
                "WR" : "Water Rescue",
                "MWR" : "Mountain or Wilderness Rescue",
                "DR" : "Disaster Rescue or Individual Tracking",
                "RES" : "Reset"
            },
            value="RES",
            inline=True,
            labelStyle={
                "padding-right" : 20
            }
        )
    ),
    html.Hr(),
    
    # Generate datatable for viewing results from the database
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i.replace('_', ' ').title(), "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        # Load results from database and stylize datatable
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[1],
        page_size=10,
        sort_action="native",
        style_table={
            'overflowX': 'auto',
        },
        style_cell={
            'textAlign': 'left',
            'paddingLeft': '10px',
            'paddingRight': '10px',
        },
        style_data={
            'width': '150px', 'minWidth': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'height': 'auto',
        },
    ),
    
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################    
# Filter datatable given chosen filter type
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Store query options for use in database
    query = {}
    
    # Filter for preferred water rescue dog breeds
    if(filter_type == "WR"):
        query["animal_type"] = "Dog"
        query["breed"] = {"$in" : ["Labrador Retriever Mix", "Chesapeak Bay Retriever", "Newfoundland"]}
        query["sex_upon_outcome"] = "Intact Female"
        query["age_upon_outcome_in_weeks"] = {"$gte": 26, "$lte": 156}
    
    # Filter for preferred mountain/wilderness rescue dog breeds
    elif(filter_type == "MWR"):
        query["animal_type"] = "Dog"
        query["breed"] = {"$in" : ["Alaskan Malamute", "German Shepherd", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}
        query["sex_upon_outcome"] = "Intact Male"
        query["age_upon_outcome_in_weeks"] = {"$gte": 26, "$lte": 156}
        
    # Filter for preferred disaster rescue/individual tracking dog breeds
    elif(filter_type == "DR"):
        query["animal_type"] = "Dog"
        query["breed"] = {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}
        query["sex_upon_outcome"] = "Intact Male"
        query["age_upon_outcome_in_weeks"] = {"$gte": 20, "$lte": 300}
        
    # Read from database given query parameters
    # Remove the _id column from resulting data
    # Return data to the datatable
    df = pd.DataFrame.from_records(db.read(query))
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    # Add frequnecy of breed to each entry
    data = pd.DataFrame.from_dict(viewData)
    data["frequency"] = data.groupby("breed")["breed"].transform("count")
    
    # Get minimum breed frequency to filter by
    breedFreqs = data["breed"].value_counts()
    minFreq = breedFreqs.head(10).iloc[-1]
    
    # Change entries with breed names under minimum frequency to 'other'
    data.loc[(data["frequency"] < minFreq), "breed"] = "other"
    
    # Change entries with null breed names to 'other'
    data.loc[(data["breed"].isnull()), "breed"] = "other"
    
    # Return pie graph of breed frequency
    return [
        dcc.Graph(            
            figure = px.pie(data, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a selected row and/or single data point
#on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'),
    Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_styles(selected_columns, selected_rows):
    # List to be returned that contains conditional styling
    styles = []
    
    # Highlight selected column box
    for col in selected_columns or []:
        styles.append({
            'if': { 'column_id': col },
            'background_color': '#D2F3FF'
        })
        
    # Highlight selected row
    for row in selected_rows or []:
        styles.append({
            'if': { 'row_index': row },
            'background_color': '#BBFFBB'
        })
    
    # Return conditional styling
    return styles

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://morrislaser-veronahistory-3000.codio.io/proxy/8050/
